In [0]:
%pylab inline

In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [0]:
cleaned_tweets_encryption = dataiku.Dataset("cleaned_tweets_encryption")
cleaned_tweets_encryption_df = cleaned_tweets_encryption.get_dataframe()

## 8.1. Retrait des duplicatas

In [0]:
# Remove duplicates based on the 'is_duplicated' column
cleaned_tweets_encryption_df = cleaned_tweets_encryption_df[cleaned_tweets_encryption_df['is_duplicated'] == 0]

## 8.2. Filter to balance the data on label column

In [0]:
cleaned_tweets_encryption_df['label'].value_counts()

In [0]:
min_count = cleaned_tweets_encryption_df['label'].value_counts().min()
balanced_df = cleaned_tweets_encryption_df.groupby('label', group_keys=False).apply(lambda x: x.sample(min(len(x), min_count))).reset_index(drop=True)


In [0]:
cleaned_tweets_encryption_df['label'].value_counts()

## 8.3. Stemming

In [0]:
# Apply stemming to the 'text' column
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def apply_stemming(text):
    words = str(text).split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

balanced_df['text'] = balanced_df['text'].apply(apply_stemming)

In [0]:
# Recipe outputs
prepared_tweets_encryption = dataiku.Dataset("prepared_tweets_encryption")
prepared_tweets_encryption.write_with_schema(balanced_df)